# 🌳 Dendritic YOLOv8 - PATCHED (No Debugger Issues!)

This notebook runs YOLOv8 with dendritic optimization - **COMPLETELY NON-INTERACTIVE**

## What's Fixed
- ✅ Debugger completely disabled (no more pdb prompts!)
- ✅ Runs fully automatically
- ✅ Generates PAI.png without interaction

## Setup
1. **Runtime → Change runtime type → GPU (T4 or L4)**
2. **Run all cells** (no manual interaction needed!)
3. Training takes ~10-15 minutes on GPU

---

## Cell 1: Install Dependencies

In [ ]:
# Install required packages
!pip install -q ultralytics
!pip install -q perforatedai==3.0.7

print("\n✓ Packages installed successfully!")
print("⚠️  Please wait 5 seconds before running next cell...")

## Cell 2: Verify Installation

In [ ]:
# Verify everything is working
import torch
import sys

print("="*70)
print("SYSTEM CHECK")
print("="*70)
print(f"✓ Python: {sys.version.split()[0]}")
print(f"✓ PyTorch: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ CUDA version: {torch.version.cuda}")
else:
    print("⚠️  No GPU detected - training will be VERY slow!")
    print("   Go to: Runtime → Change runtime type → T4 GPU")

# Try importing perforatedai
try:
    import perforatedai
    print(f"✓ PerforatedAI: 3.0.7")
except ImportError as e:
    print(f"❌ PerforatedAI import failed: {e}")
    print("   Try: Runtime → Restart runtime, then run cells again")

print("="*70)

## Cell 3: Download Patched Training Script

In [ ]:
# Create the patched training script directly
script_content = '''#!/usr/bin/env python3
"""Patched YOLOv8 Dendritic Training - NO DEBUGGER ISSUES"""

import sys
import os

# CRITICAL: Disable debugger BEFORE any imports
import pdb
import bdb

def noop(*args, **kwargs):
    pass

pdb.set_trace = noop
pdb.Pdb.set_trace = noop
bdb.Bdb.set_trace = noop

try:
    import ipdb
    ipdb.set_trace = noop
except ImportError:
    pass

print("="*70)
print("DENDRITIC YOLOV8 - PATCHED VERSION")
print("Debugger disabled - runs non-interactively")
print("="*70)

import torch
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from ultralytics import YOLO
from perforatedai import globals_perforatedai as GPA
from perforatedai import utils_perforatedai as UPA

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}\n")

# Load model
yolo = YOLO("yolov8n.pt")
model = yolo.model
baseline_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Baseline: {baseline_params/1e6:.2f}M params\n")

# Save input stem
input_stem = model.model[0]

# Configure PAI
GPA.pc.set_testing_dendrite_capacity(False)
GPA.pc.set_verbose(True)
GPA.pc.set_unwrapped_modules_confirmed(True)

# Initialize PAI
print("Initializing PerforatedAI...")
model = UPA.initialize_pai(model, doing_pai=True, save_name="PAI", maximizing_score=True)
model.model[0] = input_stem
model = model.to(device)
print("[OK] Initialized\n")

# Setup optimizer
GPA.pai_tracker.set_optimizer(optim.Adam)
GPA.pai_tracker.set_scheduler(ReduceLROnPlateau)
optimArgs = {\'params\': model.parameters(), \'lr\': 0.001}
schedArgs = {\'mode\': \'max\', \'patience\': 3}
optimizer, scheduler = GPA.pai_tracker.setup_optimizer(model, optimArgs, schedArgs)

# Training loop
print("="*70)
print("TRAINING (5 epochs)")
print("="*70)

epoch = 0
training_complete = False

while not training_complete and epoch < 5:
    print(f"\nEpoch {epoch+1}/5")
    
    yolo.model = model
    results = yolo.train(data=\'coco128.yaml\', epochs=1, imgsz=640, batch=16, 
                        device=device, exist_ok=True, verbose=False, 
                        project=\'runs/train\', name=\'dendritic\')
    model = yolo.model
    
    train_score = float(results.results_dict.get(\'metrics/mAP50(B)\', 0))
    GPA.pai_tracker.add_extra_score(train_score * 100, \'train\')
    
    model.eval()
    yolo.model = model
    val_results = yolo.val(data="coco128.yaml", verbose=False)
    val_score = float(val_results.box.map50)
    
    print(f"  Train: {train_score:.4f}, Val: {val_score:.4f}")
    
    model, restructured, training_complete = GPA.pai_tracker.add_validation_score(
        val_score * 100, model)
    model = model.to(device)
    yolo.model = model
    
    if restructured:
        print("  >>> DENDRITES ADDED! <<<")
        optimArgs[\'params\'] = model.parameters()
        optimizer, scheduler = GPA.pai_tracker.setup_optimizer(model, optimArgs, schedArgs)
    
    epoch += 1

# Save graphs
print("\nSaving graphs...")
try:
    GPA.pai_tracker.save_graphs()
    print("[OK] PAI.png saved!")
except Exception as e:
    print(f"Error: {e}")

final_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nFinal: {final_params/1e6:.2f}M params")
print("Training complete!")
'''

with open('train_patched.py', 'w') as f:
    f.write(script_content)

print("✓ Patched training script created")
print("  This script completely disables the debugger")
print("  No manual interaction required!")

## Cell 4: Run Training

This will:
- Train YOLOv8n with dendritic optimization
- Automatically add dendrites when performance plateaus
- Generate the PAI graph showing dendrite effects
- Save results to `PAI/PAI.png`

**Expected time: 10-15 minutes on T4 GPU**

**NO MANUAL INTERACTION NEEDED!**

In [ ]:
# Run the training script
!python train_patched.py

## Cell 5: View Results

In [ ]:
import os
from IPython.display import Image, display

print("="*70)
print("CHECKING RESULTS")
print("="*70)

# Check for PAI directory
if os.path.exists('PAI'):
    print("\n✓ PAI directory found")
    print("\n📁 Generated files:")
    for f in sorted(os.listdir('PAI')):
        size = os.path.getsize(f'PAI/{f}')
        print(f"  - PAI/{f} ({size:,} bytes)")
else:
    print("\n❌ PAI directory not found")

# Display the graph
if os.path.exists('PAI/PAI.png'):
    print("\n" + "="*70)
    print("📈 PERFORATEDAI OUTPUT GRAPH")
    print("="*70)
    
    display(Image(filename='PAI/PAI.png'))
    
    print("\n" + "="*70)
    print("GRAPH INTERPRETATION")
    print("="*70)
    print("📈 Green line:    Training scores")
    print("📊 Orange line:   Validation scores")
    print("📍 Vertical bars: When dendrites were added")
    print("="*70)
else:
    print("\n❌ PAI/PAI.png not found")

## Cell 6: Download Results

In [ ]:
from google.colab import files
import os

print("📥 Downloading files...\n")

downloaded = 0

# Download PAI graph
if os.path.exists('PAI/PAI.png'):
    try:
        files.download('PAI/PAI.png')
        print("✓ Downloaded: PAI/PAI.png (REQUIRED)")
        downloaded += 1
    except Exception as e:
        print(f"❌ Failed: {e}")
else:
    print("❌ PAI/PAI.png not found")

# Download CSV files
if os.path.exists('PAI'):
    for f in os.listdir('PAI'):
        if f.endswith('.csv'):
            try:
                files.download(f'PAI/{f}')
                print(f"✓ Downloaded: PAI/{f}")
                downloaded += 1
            except Exception as e:
                print(f"❌ Failed: {e}")

print(f"\n✓ Downloaded {downloaded} file(s)!")
if downloaded > 0:
    print("Check your browser downloads folder.")
else:
    print("⚠️  No files downloaded")

---

## ✅ Success!

You should now have:
- PAI.png downloaded to your computer
- Graph showing training/validation curves with dendrite markers
- Ready to commit to your PR!

---

**Project:** Dendritic YOLOv8 for Edge Object Detection

**Team:** Will Wild - woakwild@gmail.com - https://github.com/wildhash

**PerforatedAI:** https://github.com/PerforatedAI/PerforatedAI